---
# --- Day 17: Pyroclastic Flow ---
---

In [1]:
import numpy as np

In [2]:
V = lambda x: np.array(x)

## Load data

In [3]:
full_puzzle_data = True

In [4]:
file_suffix = "" if full_puzzle_data else "_test"
with open(f"data/day17_input{file_suffix}.txt", "r") as f:
    JET_PATTERN = f.read()

In [5]:
rocks = [
    [
        "..####."
    ],
    [
        "...#...",
        "..###..",
        "...#..."
    ],
    [
        "....#..",
        "....#..",
        "..###.."
    ],
    [
        "..#....",
        "..#....",
        "..#....",
        "..#...."
    ],
    [
        "..##...",
        "..##..."
    ]
]

In [6]:
ROCKS_MATRICES = [
    V([[int(el=="#") for el in r] for r in rock])
    for rock in rocks       
]

## --- Part One ---

In [7]:
def print_tower(tower: np.ndarray) -> None:
    for row in tower:
        print("".join(["." if px == 0 else "#" for px in row]))

In [8]:
def no_collision_present(a: np.ndarray) -> bool:
    return np.amax(a) == 1

def settle_a_rock(rock: np.ndarray, tower: np.ndarray, jet_step: int) -> (np.ndarray, int):
    rock_height = rock.shape[0]
    tower = np.vstack((np.zeros((rock_height+3,7), dtype=int), tower))
    i = 0
    landed = False
    
    while not landed:
        # push by the jet of gas
        jet_symbol = JET_PATTERN[jet_step % len(JET_PATTERN)]
        temp_rock = None
        if jet_symbol == "<": # left
            if not rock[:, 0].sum() > 0: # already at the wall
                temp_rock = np.hstack((rock[:,1:], rock[:,0].reshape(-1, 1)))
        elif jet_symbol == ">": # right
            if not rock[:, -1].sum() > 0: # already at the wall
                temp_rock = np.hstack((rock[:,-1].reshape(-1, 1), rock[:,:-1]))
        else:
            raise ValueError("I cannot interpret the symbol.")
        if temp_rock is not None:
            temp_strip = tower[i:i+rock_height,:] + temp_rock
            if no_collision_present(temp_strip):
                rock = temp_rock
        jet_step += 1
                
        # downward movement
        temp_strip = tower[i+1:i+1+rock_height,:] + rock
        if no_collision_present(temp_strip):
            i += 1
        else:
            landed = True        
        
    tower[i:i+rock_height] += rock
    tower = tower[np.sum(tower, axis=1)>0, :] # remove empty line on top
    jet_step = jet_step % len(JET_PATTERN)
    return tower, jet_step

def build_tower(number_of_rocks: int, jet_step: int=0, initial_tower: np.ndarray=None) -> np.ndarray:
    tower = initial_tower if initial_tower is not None else np.ones((1,7), dtype=int)
    
    for i in range(number_of_rocks):
        rock = ROCKS_MATRICES[i % len(ROCKS_MATRICES)]
        tower, jet_step = settle_a_rock(rock, tower, jet_step)
    
    return tower

In [9]:
tower = build_tower(2022)

In [10]:
tower_height = tower.shape[0] - 1
print(tower_height)

3100


## --- Part Two ---

In [11]:
def compute_tower_top_signature(top: np.ndarray) -> int:
    return int("".join(["".join([f"{b}" for b in row]) for row in top]), 2)

def revert_tower_top_signature(s: int) -> np.ndarray:
    tower_base_string = bin(s)[2:].rjust(28, "0")
    tower_base = V([list(map(int, tower_base_string[i*7:(i+1)*7])) for i in range(4)])
    return tower_base

In [12]:
initial_heights = []
signatures = {}

tower = np.ones((1,7), dtype=int)
i = 0
jet_step = 0
common_signature = -1
while (common_signature == -1) and (i < 20000):
    rock = ROCKS_MATRICES[i % len(ROCKS_MATRICES)]
    
    #initial signature & tower heights
    if i % len(ROCKS_MATRICES) == 0:
        iter_signature = (compute_tower_top_signature(tower[:4,:]), jet_step)
        if iter_signature in signatures:
            common_signature = iter_signature
            print(f"Common signature <{common_signature}> found!")
        signatures[iter_signature] = signatures.get(iter_signature, []) + [i]
    initial_heights.append(tower.shape[0] -1)
    
    tower, jet_step = settle_a_rock(rock, tower, jet_step)    
    i += 1

Common signature <(101451792, 824)> found!


In [13]:
n = 1000000000000

first_app = signatures[common_signature][0]
second_app = signatures[common_signature][1]
period = second_app - first_app
initial_height = initial_heights[first_app]
height_uplift = initial_heights[second_app] - initial_height

period_count = (n - first_app) // period
last_app = first_app + period * period_count
height_at_last = initial_height + period_count * height_uplift
print(f"The height at the {last_app} iteration is: {height_at_last}.")

The height at the 1000000000000 iteration is: 1540634005751.


In [14]:
tower_top = revert_tower_top_signature(common_signature[0])
jet_step = common_signature[1]

final_tower = build_tower(n - last_app, jet_step=jet_step, initial_tower=tower_top)
extra_height = final_tower.shape[0] - 4

In [15]:
print(height_at_last + extra_height)

1540634005751
